# Introduction

### Okay, so hopefully this is a not terrible way to look through all this code. I start with filling NA values with means. I still have to do the similarity score thing. Also, I turn the long list of nationalities into just North American and European skaters.

In [18]:
import pandas as pd
pd.options.mode.chained_assignment = None

desired_width = 100
pd.set_option('display.width', desired_width)

df = pd.read_csv('rftest.csv',encoding='latin-1')
df = df.fillna(df.mean())
df['NAEU'] = 0
df.loc[df['NAT'] == 'CA', 'NAEU'] = 1
df.loc[df['NAT'] == 'US', 'NAEU'] = 1

print(df.head())

    EPID  YEAR  OVERALL                 TEAM          PLAYER NAT POS POSFDG  \
0   9223  2009        1   New York Islanders    John Tavares  CA   C      F   
1   6007  2009        2  Tampa Bay Lightning   Victor Hedman  SE   D      D   
2  12571  2009        3   Colorado Avalanche    Matt Duchene  CA   C      F   
3  14657  2009        4    Atlanta Thrashers    Evander Kane  CA  LW      F   
4  14739  2009        5    Los Angeles Kings  Brayden Schenn  CA   C      F   

                                                 URL  TYPE  ...   PDY_LEAGUE  PDY_GP  PDY_G  \
0  http://www.hockey-reference.com/players/t/tava...  NA.S  ...          OHL      59   40.0   
1  http://www.hockey-reference.com/players/h/hedm...  EU.S  ...          SHL      39    2.0   
2  http://www.hockey-reference.com/players/d/duch...  NA.S  ...          OHL      64   30.0   
3  http://www.hockey-reference.com/players/k/kane...  NA.S  ...          WHL      65   24.0   
4  http://www.hockey-reference.com/players/s/sche.

### Now we run a regression real quick. As expected, CSS rank is very important. Stuff like age and nationality don't really do much, so I just left them in. The R-squared always seems to hover around 50%. As discussed before, the better teammate predictor and the "superleagues" don't help at all. This info is seemingly contained within CSS rank.

In [19]:
import statsmodels.formula.api as sm
import numpy as np

result = sm.ols(formula="OVERALL ~ TYPE * RANK + NAEU + POSFDG + HT + WT + AGE + DY_PPG", data=df).fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                OVERALL   R-squared:                       0.503
Model:                            OLS   Adj. R-squared:                  0.496
Method:                 Least Squares   F-statistic:                     71.44
Date:                Mon, 11 Sep 2017   Prob (F-statistic):          5.80e-149
Time:                        04:41:39   Log-Likelihood:                -5592.9
No. Observations:                1075   AIC:                         1.122e+04
Df Residuals:                    1059   BIC:                         1.130e+04
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept           187.6955     64.94

### Just looking at the predictions, it's obvious they need to be rescaled.

In [20]:
df['PREDOLS'] = result.predict()
df1 = df[['YEAR','OVERALL','PLAYER','PREDOLS']]
df1 = df1.sort_values(by='PREDOLS', ascending = True)
print(df1.head(20))

     YEAR  OVERALL              PLAYER    PREDOLS
632  2006        1        Erik Johnson  17.508196
845  2005        1       Sidney Crosby  20.511482
633  2006        2        Jordan Staal  23.828401
649  2006       18       Chris Stewart  25.178279
211  2008        2        Drew Doughty  25.180621
846  2005        2          Bobby Ryan  25.210316
221  2008       12         Tyler Myers  26.442303
62   2009       63        Ben Hanowski  26.968887
8    2009        9         Jared Cowen  27.035376
212  2008        3       Zach Bogosian  27.219266
213  2008        4    Alex Pietrangelo  27.326010
0    2009        1        John Tavares  27.530370
422  2007        2  James van Riemsdyk  27.756304
214  2008        5         Luke Schenn  28.579353
423  2007        3         Kyle Turris  28.830935
18   2009       19       Chris Kreider  29.181108
425  2007        5         Karl Alzner  29.451442
426  2007        6          Sam Gagner  29.493511
40   2009       41         Zach Budish  29.867714


### Have to rescale by year. Here's an example from 2007. The RMSE of the rescaled difference is bad.

In [21]:
year = 2007

df1 = df[df.YEAR == year]
df1 = df1.sort_values(by='PREDOLS', ascending = True)
df1['MYRANK'] = range(len(df1))
df1['MYRANK'] = df1['MYRANK'] + 1
df1 = df1.sort_values(by='OVERALL', ascending = True)
df1['REALRANK'] = range(len(df1))
df1['REALRANK'] = df1['REALRANK'] + 1
df1['DIFF'] = df1['MYRANK'] - df1['REALRANK']
df1 = df1.sort_values(by='PREDOLS', ascending = True)

print(df1[['PLAYER','MYRANK','REALRANK','DIFF']].head(20))

rmse = (np.mean((df1.MYRANK - df1.REALRANK)**2))**(1/2)
print('RMSE')
print(rmse)

                 PLAYER  MYRANK  REALRANK  DIFF
422  James van Riemsdyk       1         2    -1
423         Kyle Turris       2         3    -1
425         Karl Alzner       3         5    -2
426          Sam Gagner       4         6    -2
432       Ryan McDonagh       5        12    -7
421        Patrick Kane       6         1     5
430      Keaton Ellerby       7        10    -3
455         Tommy Cross       8        35   -27
452       Brett Maclean       9        32   -23
427       Jakub Voracek      10         7     3
440     Angelo Esposito      11        20    -9
442      Max Pacioretty      12        22   -10
448   Nicholas Petrecki      13        28   -15
445       Patrick White      14        25   -11
465         Colby Cohen      15        45   -30
429       Logan Couture      16         9     7
428         Zach Hamill      17         8     9
446        David Perron      18        26    -8
450           Nick Ross      19        30   -11
467         Dana Tyrell      20        4

### Now we got some random forest stuff. Train on 4 years of data, test on 1. The RMSE of the testing set is still pretty bad.

In [22]:
from sklearn.ensemble import RandomForestRegressor
import sklearn.metrics as skm

dummies = pd.get_dummies(df[['TYPE','POSFDG']])
df2 = pd.concat([df,dummies],axis=1)

year = 2007
dftest = df2[df2.YEAR == year]
dftrain = df2[df2.YEAR != year]

cols = ['RANK','NAEU','HT','WT','AGE','DY_PPG','TYPE_NA.S','TYPE_EU.S','TYPE_NA.G','TYPE_EU.G','TYPE_UNR',
        'POSFDG_F','POSFDG_D','POSFDG_G']
rf = RandomForestRegressor(n_estimators=100)
rf.fit(dftrain[cols], dftrain.OVERALL)

r2 = skm.r2_score(dftrain.OVERALL, rf.predict(dftrain[cols]))
rmse = (np.mean((dftrain.OVERALL - rf.predict(dftrain[cols]))**2))**(1/2)
print('Training R2')
print(r2)
print('Training RMSE')
print(rmse)

dftest['PREDRF'] = rf.predict(dftest[cols])

dftest = dftest.sort_values(by='PREDRF', ascending = True)
dftest['MYRANK'] = range(len(dftest))
dftest['MYRANK'] = dftest['MYRANK'] + 1
dftest = dftest.sort_values(by='OVERALL', ascending = True)
dftest['REALRANK'] = range(len(dftest))
dftest['REALRANK'] = dftest['REALRANK'] + 1
dftest['DIFF'] = dftest['MYRANK'] - dftest['REALRANK']
dftest = dftest.sort_values(by='PREDRF', ascending = True)

print(dftest[['PLAYER','MYRANK','REALRANK','DIFF']].head(20))
rmse = (np.mean((dftest.MYRANK - dftest.REALRANK)**2))**(1/2)
print('Testing RMSE')
print(rmse)

Training R2
0.932533005964
Training RMSE
16.2917128871
                 PLAYER  MYRANK  REALRANK  DIFF
423         Kyle Turris       1         3    -2
422  James van Riemsdyk       2         2     0
421        Patrick Kane       3         1     2
425         Karl Alzner       4         5    -1
440     Angelo Esposito       5        20   -15
430      Keaton Ellerby       6        10    -4
452       Brett Maclean       7        32   -25
446        David Perron       8        26   -18
428         Zach Hamill       9         8     1
437   Alexei Cherepanov      10        17    -7
426          Sam Gagner      11         6     5
429       Logan Couture      12         9     3
432       Ryan McDonagh      13        12     1
427       Jakub Voracek      14         7     7
457       Stefan Legein      15        37   -22
433          Lars Eller      16        13     3
445       Patrick White      17        25    -8
455         Tommy Cross      18        35   -17
442      Max Pacioretty      19  

### Okay, now do all of the above with NHL value (Point Shares). Does not look great so far...

In [23]:
result = sm.ols(formula="NHL_PS ~ TYPE * RANK + NAEU + POSFDG + HT + WT + AGE + DY_PPG", data=df).fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                 NHL_PS   R-squared:                       0.084
Model:                            OLS   Adj. R-squared:                  0.071
Method:                 Least Squares   F-statistic:                     6.481
Date:                Mon, 11 Sep 2017   Prob (F-statistic):           2.13e-13
Time:                        04:41:58   Log-Likelihood:                -4167.4
No. Observations:                1075   AIC:                             8367.
Df Residuals:                    1059   BIC:                             8446.
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            38.7484     17.24

### But straight up draft rank doesn't predict NHL value amazingly either, so that's a relief. Although once we include a quadratic trend, the R2 becomes a bit better.

In [24]:
df['OVERALL2'] = df['OVERALL']**2
result = sm.ols(formula="NHL_PS ~ OVERALL + OVERALL2", data=df).fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                 NHL_PS   R-squared:                       0.126
Model:                            OLS   Adj. R-squared:                  0.125
Method:                 Least Squares   F-statistic:                     77.46
Date:                Mon, 11 Sep 2017   Prob (F-statistic):           3.78e-32
Time:                        04:42:07   Log-Likelihood:                -4142.1
No. Observations:                1075   AIC:                             8290.
Df Residuals:                    1072   BIC:                             8305.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     23.9418      1.049     22.830      0.0

### Random forest. There are hits and misses. Most notably, I did a better job of seeing Claude Giroux's value than the scouts did in '06. But the rank stuff is kind of misleading because like 60% of players basically should be ranked last because they didn't make it to the NHL.

In [25]:
dummies = pd.get_dummies(df[['TYPE','POSFDG']])
df3 = pd.concat([df,dummies],axis=1)

year = 2006
dftestvalue = df3[df3.YEAR == year]
dftrainvalue = df3[df3.YEAR != year]

cols = ['RANK','NAEU','HT','WT','AGE','DY_PPG','TYPE_NA.S','TYPE_EU.S','TYPE_NA.G','TYPE_EU.G','TYPE_UNR',
        'POSFDG_F','POSFDG_D','POSFDG_G','OVERALL']
rf = RandomForestRegressor(n_estimators=100)
rf.fit(dftrainvalue[cols], dftrainvalue.NHL_PS)

r2 = skm.r2_score(dftrainvalue.NHL_PS, rf.predict(dftrainvalue[cols]))
rmse = (np.mean((dftrainvalue.NHL_PS - rf.predict(dftrainvalue[cols]))**2))**(1/2)
print('Training R2')
print(r2)
print('Training RMSE')
print(rmse)

dftestvalue['PREDRF'] = rf.predict(dftestvalue[cols])

dftestvalue = dftestvalue.sort_values(by='PREDRF', ascending = False)
dftestvalue['MYRANK'] = range(len(dftestvalue))
dftestvalue['MYRANK'] = dftestvalue['MYRANK'] + 1
dftestvalue = dftestvalue.sort_values(by='NHL_PS', ascending = False)
dftestvalue['REALRANK'] = range(len(dftestvalue))
dftestvalue['REALRANK'] = dftestvalue['REALRANK'] + 1
dftestvalue['DIFF'] = dftestvalue['MYRANK'] - dftestvalue['REALRANK']
dftestvalue = dftestvalue.sort_values(by='PREDRF', ascending = False)

print(dftestvalue[['PLAYER','MYRANK','REALRANK','DIFF']].head(20))
rmse = (np.mean((dftestvalue.MYRANK - dftestvalue.REALRANK)**2))**(1/2)
print('Testing RMSE')
print(rmse)

Training R2
0.883853913589
Training RMSE
4.1210361269
                PLAYER  MYRANK  REALRANK  DIFF
632       Erik Johnson       1        12   -11
633       Jordan Staal       2         9    -7
642   Jonathan Bernier       3        13   -10
635  Nicklas Backstrom       4         2     2
654    Semyon Varlamov       5         5     0
646      Riku Helenius       6       206  -200
634     Jonathan Toews       7         1     6
653      Claude Giroux       8         6     2
636        Phil Kessel       9         4     5
658   Ivan Vishnevskiy      10       188  -178
668    Yury Alexandrov      11       125  -114
686       Denis Bodrov      12       126  -114
643       Bryan Little      13        10     3
649      Chris Stewart      14        18    -4
655     Dennis Persson      15       127  -112
705       Jeff Zatkoff      16       165  -149
665    Michal Neuvirth      17        16     1
638        Kyle Okposo      18        14     4
733        Kyle Medvec      19       128  -109
675   

### That's it. This format is great but I am not doing it justice but I can do more things between now and Wednesday.